In [8]:
import pyodbc
import pandas as pd
from pathlib import Path
import glob
import datetime
import re
import os
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from datetime import date
import math
import time

In [9]:
# home and time
home = Path.home()
todaystr = date.today().strftime('%Y-%m-%d')
# PNFV_alternative = pd.read_excel(Path(home, 'HP Inc', 'GPS TW Innovation - Documents', 'Users', 'GPS', 'PNFV', 'alternative.xlsx'))
# PNFV = pd.read_excel(Path(home, 'HP Inc', 'GPS TW Innovation - Documents', 'Users', 'GPS', 'PN FV description mapping table_ALL.xlsx'))
"""
currently the folder for buyer to update is still in GPSTW SOP - 2021 日新
REMEMBER to change the path when the folder officailly changed to GPS TW Innovation
"""
PNFV = pd.read_excel(Path(home, 'HP Inc', 'GPSTW SOP - 2021 日新', 'PN FV description mapping table_ALL.xlsx'))
# PNFV = pd.merge(PNFV, PNFV_alternative, on = 'Descr', how = 'left')

In [10]:
len(PNFV)

5536

In [4]:
start_time = time.time()
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

cursor.execute(f"SELECT COUNT(*) FROM OPS.GPS_tbl_ops_PN_FV")
conn.commit()

cursor.execute(f"DELETE FROM OPS.GPS_tbl_ops_PN_FV")
conn.commit()
print("%s seconds ---" % (time.time() - start_time))

cursor.execute(f"SELECT COUNT(*) FROM OPS.GPS_tbl_ops_PN_FV")
conn.commit()

for index, row in PNFV.iterrows():

    Commodity = str(row['Commodity'])
    Supplier = str(row['Supplier'])
    PN = str(row['PN'])
    Descr = str(row['Descr'])
    alternative = str(row['alternative part flag'])

    cursor.execute(f"INSERT INTO CSI.OPS.GPS_tbl_ops_PN_FV ( Commodity, Supplier, PN, Descr, [alternative part flag] )\
                    VALUES('{Commodity}','{Supplier}','{PN}','{Descr}','{alternative}')")
    
    print("%s seconds ---" % (time.time() - start_time))
conn.commit()
conn.close()
print("%s seconds ---" % (time.time() - start_time))

4.429688930511475 seconds ---
5.250299453735352 seconds ---
5.52035117149353 seconds ---
5.7882256507873535 seconds ---
6.055837392807007 seconds ---
6.323397636413574 seconds ---
6.595641374588013 seconds ---
6.872420072555542 seconds ---
7.145529508590698 seconds ---
7.413445711135864 seconds ---
7.680807113647461 seconds ---
7.956327676773071 seconds ---
8.227213382720947 seconds ---
8.495870113372803 seconds ---
8.787542819976807 seconds ---
9.085174322128296 seconds ---
9.361807584762573 seconds ---
9.63089108467102 seconds ---
9.900243043899536 seconds ---
10.169857263565063 seconds ---
10.437891960144043 seconds ---
10.704949378967285 seconds ---
10.974244594573975 seconds ---
11.24135947227478 seconds ---
11.509437322616577 seconds ---
11.793463706970215 seconds ---
12.06401777267456 seconds ---
12.330704927444458 seconds ---
12.600249290466309 seconds ---
12.868712663650513 seconds ---
13.143219709396362 seconds ---
13.411301612854004 seconds ---
13.677499771118164 seconds ---